In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # GPU 사용 가능 여부 확인

2.8.0
False


In [33]:
import pandas as pd
import numpy as np

In [35]:
smoking_path = "11_data/Smoking Status.csv"
df = pd.read_csv(smoking_path)

In [37]:
df.columns

Index(['Unnamed: 0', 'Date', 'Study', 'Study Link', 'Journal', 'Severe',
       'Severe lower bound', 'Severe upper bound', 'Severe p-value',
       'Severe Significant', 'Severe Adjusted', 'Severe Calculated',
       'Fatality', 'Fatality lower bound', 'Fatality upper bound',
       'Fatality p-value', 'Fatality Significant', 'Fatality Adjusted',
       'Fatality Calculated', 'Multivariate adjustment', 'Study Type',
       'Sample Size', 'Study Population', 'Added on', 'Critical only',
       'Discharged vs. death?'],
      dtype='object')

In [39]:
#2) 각 엔드포인트(Severe, Fatality)별 log(OR) 계산

In [48]:
def build_endpoint(df, prefix):
    lo = df[f"{prefix} lower bound"].astype(float)   # Convert the lower bound column to float type
    hi = df[f"{prefix} upper bound"].astype(float)   # Convert the upper bound column to float type
    pv = pd.to_numeric(df.get(f"{prefix} p-value"), errors="coerce")  # Convert p-value column to numeric, invalid values become NaN

    # Calculate log(OR): Approximate the point estimate as sqrt(lower * upper), then take the natural log
    log_or = np.log(np.sqrt(lo * hi))

    # Calculate standard error (SE) from 95% CI formula
    se = (np.log(hi) - np.log(lo)) / (2*1.96)

    # Variance = SE²
    var = se**2

    # Return as a new DataFrame
    return pd.DataFrame({
        "lower": lo,
        "upper": hi,
        "pvalue": pv,
        "log_or": log_or,
        "se": se,
        "var": var
    }).dropna(subset=["log_or","var"])  # Drop rows where log_or or var is NaN


In [49]:
#3) pooling func
"""
 fixed-effect model : Assumption: All studies share the same true effect.
 → Any differences between study results are purely due to sampling error.
Advantages:
Simple to calculate, stable with small sample sizes.
Appropriate when study results are highly consistent (low heterogeneity).
Disadvantages:
Can be biased if heterogeneity is high.


 random-effects model: The true effect may vary across studies.
Assumption: The true effect may vary across studies.
→ Differences can come from variations in participants, settings, or study designs.
Advantages:
Accounts for between-study variance (τ²), making it more realistic.
Wider confidence intervals → more conservative estimates.
Disadvantages:
Less stable with small sample sizes.
If heterogeneity is very low, it may unnecessarily widen the CI.
"""



'\n fixed-effect model : Assumption: All studies share the same true effect.\n → Any differences between study results are purely due to sampling error.\nAdvantages:\nSimple to calculate, stable with small sample sizes.\nAppropriate when study results are highly consistent (low heterogeneity).\nDisadvantages:\nCan be biased if heterogeneity is high.\n\n\n random-effects model: The true effect may vary across studies.\nAssumption: The true effect may vary across studies.\n→ Differences can come from variations in participants, settings, or study designs.\nAdvantages:\nAccounts for between-study variance (τ²), making it more realistic.\nWider confidence intervals → more conservative estimates.\nDisadvantages:\nLess stable with small sample sizes.\nIf heterogeneity is very low, it may unnecessarily widen the CI.\n'

In [50]:
def pooled_fixed(log_or, var):
    # 1. Calculate weights for each study: inverse of variance
    #    Smaller variance (more precise estimate) → larger weight.
    w = 1.0 / var

    # 2. Compute the weighted average of log(OR)
    #    Sum of (log(OR) * weight) divided by total weight.
    est = np.sum(w * log_or) / np.sum(w)

    # 3. Calculate the overall Standard Error (SE)
    #    SE = sqrt(1 / sum of weights)
    se = np.sqrt(1.0 / np.sum(w))

    # 4. Calculate the 95% Confidence Interval (CI) on the log scale
    #    CI = estimate ± 1.96 * SE  (1.96 is the z-score for 95% CI)
    ci = (est - 1.96 * se, est + 1.96 * se)

    # 5. Return: pooled log(OR) estimate and its 95% CI
    return est, ci

In [51]:
def pooled_random_DL(log_or, var):
    # 1. Initial weights (fixed-effect weights) = inverse of variance
    w = 1.0 / var

    # 2. Number of studies
    k = len(log_or)

    # 3. Fixed-effect pooled estimate of log(OR)
    est_fixed = np.sum(w * log_or) / np.sum(w)

    # 4. Cochran’s Q statistic: measures total heterogeneity across studies
    Q = np.sum(w * (log_or - est_fixed) ** 2)

    # 5. c is a constant used in the DerSimonian–Laird tau² formula
    c = np.sum(w) - (np.sum(w ** 2) / np.sum(w))

    # 6. Between-study variance (tau²) using the DerSimonian–Laird method
    #    If tau² < 0, set to 0 (no between-study variance).
    tau2 = max(0.0, (Q - (k - 1)) / c) if k > 1 else 0.0

    # 7. Random-effects weights: incorporate both within-study variance and between-study variance (tau²)
    w_star = 1.0 / (var + tau2)

    # 8. Random-effects pooled estimate of log(OR)
    est = np.sum(w_star * log_or) / np.sum(w_star)

    # 9. Standard Error (SE) of the pooled estimate
    se = np.sqrt(1.0 / np.sum(w_star))

    # 10. 95% Confidence Interval (log scale)
    ci = (est - 1.96 * se, est + 1.96 * se)

    # 11. I² statistic: percentage of total variation due to between-study heterogeneity
    I2 = max(0.0, (Q - (k - 1)) / Q) * 100 if k > 1 and Q > (k - 1) else 0.0

    # Return: pooled log(OR), 95% CI, tau², and I²
    return est, ci, tau2, I2


In [43]:
#4) Severe & Fatality 결과 출력

In [52]:
for ep in ["Severe", "Fatality"]:
    # 1. Build the endpoint DataFrame (lower, upper, p-value, log_or, SE, var)
    ep_df = build_endpoint(df, ep)
    
    # 2. Skip if there is no usable data for this endpoint
    if len(ep_df) == 0:
        print(f"[{ep}] usable data 없음")  # "no usable data"
        continue

    # 3. Calculate the fixed-effect pooled estimate (log scale)
    est_f, ci_f = pooled_fixed(ep_df["log_or"], ep_df["var"])

    # 4. Calculate the random-effects pooled estimate (log scale)
    est_r, ci_r, tau2, I2 = pooled_random_DL(ep_df["log_or"], ep_df["var"])

    # 5. Convert the random-effects log(OR) estimate and CI back to OR scale
    or_r = np.exp(est_r)
    lo_r, hi_r = np.exp(ci_r[0]), np.exp(ci_r[1])

    # 6. Interpret the result:
    #    - If lower CI > 1 → smoking increases risk
    #    - If upper CI < 1 → smoking decreases risk
    #    - Otherwise → no clear difference
    verdict = "Smoking risk ↑" if lo_r > 1 else ("Smoking risk ↓" if hi_r < 1 else "No clear difference")

    # 7. Print the random-effects result with OR, 95% CI, I², and interpretation
    print(f"[{ep}] Random-effects OR={or_r:.2f} (95% CI {lo_r:.2f}–{hi_r:.2f}), "
          f"I²={I2:.1f}% → {verdict}")


[Severe] Random-effects OR=1.80 (95% CI 1.56–2.07), I²=68.5% → Smoking risk ↑
[Fatality] Random-effects OR=1.27 (95% CI 1.12–1.44), I²=54.1% → Smoking risk ↑


/opt/anaconda3/envs/nlp_automl_env/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
